# Code for download and save dataset from source.

In [1]:
# import urllib.request
# import gzip
# import pandas as pd

# url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Digital_Music_5.json.gz"
# file_name = "reviews_Digital_Music_5.json.gz"

# file = urllib.request.URLopener()
# file.retrieve(url, file_name)

# def parse(path):
#     g = gzip.open(path, 'rb')
#     for l in g:
#         yield eval(l)

# def getDF(path):
#     i = 0
#     df = {}
#     for d in parse(path):
#         df[i] = d
#         i += 1
#     return pd.DataFrame.from_dict(df, orient='index')

# df = getDF(file_name)
# df.to_csv("{}.csv".format(file_name))


In [2]:
import pandas as pd
import random
import string
import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk import classify
from nltk import NaiveBayesClassifier

%pprint

Pretty printing has been turned OFF


# Data Preparation.

In [3]:
df = pd.read_csv("reviews_Digital_Music_5.json.gz.csv")
df.head()

,Unnamed: 0,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came ou...",5.0,Enya's last great album,1158019200,"09 12, 2006"
1,1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Mem...",5.0,Enya at her most elegant,991526400,"06 3, 2001"
2,2,A38IRL0X2T4DPF,5555991584,bob turnley,"[2, 2]",I never thought Enya would reach the sublime h...,5.0,The best so far,1058140800,"07 14, 2003"
3,3,A22IK3I6U76GX0,5555991584,Calle,"[1, 1]",This is the third review of an irish album I w...,5.0,Ireland produces good music.,957312000,"05 3, 2000"
4,4,A1AISPOIIHTHXX,5555991584,"Cloud ""...""","[1, 1]","Enya, despite being a successful recording art...",4.0,4.5; music to dream to,1200528000,"01 17, 2008"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64706 entries, 0 to 64705
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      64706 non-null  int64  
 1   reviewerID      64706 non-null  object 
 2   asin            64706 non-null  object 
 3   reviewerName    64529 non-null  object 
 4   helpful         64706 non-null  object 
 5   reviewText      64705 non-null  object 
 6   overall         64706 non-null  float64
 7   summary         64706 non-null  object 
 8   unixReviewTime  64706 non-null  int64  
 9   reviewTime      64706 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 4.9+ MB


Drop all columns that we don't need in the analysis.

In [5]:
df.drop(labels=['Unnamed: 0', 'reviewerID', 'reviewerName', 'helpful', 'unixReviewTime', 'reviewTime'],
        axis='columns', inplace=True)

In [6]:
df.head()

,asin,reviewText,overall,summary
0,5555991584,"It's hard to believe ""Memory of Trees"" came ou...",5.0,Enya's last great album
1,5555991584,"A clasically-styled and introverted album, Mem...",5.0,Enya at her most elegant
2,5555991584,I never thought Enya would reach the sublime h...,5.0,The best so far
3,5555991584,This is the third review of an irish album I w...,5.0,Ireland produces good music.
4,5555991584,"Enya, despite being a successful recording art...",4.0,4.5; music to dream to


The overall score is from 1 to 5; we should check the percentage of each score in the dataset.

In [7]:
df['overall'].value_counts(normalize=True)

5.0    0.549872
4.0    0.255556
3.0    0.104921
2.0    0.046518
1.0    0.043134
Name: overall, dtype: float64

Print out the review text of the first review.

In [8]:
df.iloc[0,1]

'It\'s hard to believe "Memory of Trees" came out 11 years ago;it has held up well over the passage of time.It\'s Enya\'s last great album before the New Age/pop of "Amarantine" and "Day without rain." Back in 1995,Enya still had her creative spark,her own voice.I agree with the reviewer who said that this is her saddest album;it is melancholy,bittersweet,from the opening title song."Memory of Trees" is elegaic&majestic.;"Pax Deorum" sounds like it is from a Requiem Mass,it is a dark threnody.Unlike the reviewer who said that this has a "disconcerting" blend of spirituality&sensuality;,I don\'t find it disconcerting at all."Anywhere is" is a hopeful song,looking to possibilities."Hope has a place" is about love,but it is up to the listener to decide if it is romantic,platonic,etc.I\'ve always had a soft spot for this song."On my way home" is a triumphant ending about return.This is truly a masterpiece of New Age music,a must for any Enya fan!'

Print out the summary text of the first review.

In [9]:
df.iloc[0,3]

"Enya's last great album"

Combine the review text column and summary column into one colum.

In [10]:
df['reviewText'] = df['reviewText'] + ' ' + df['summary']
df.drop(labels=['summary'], axis='columns', inplace=True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64705 entries, 0 to 64705
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   asin        64705 non-null  object 
 1   reviewText  64705 non-null  object 
 2   overall     64705 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.0+ MB


We label the sentiment for each review in the dataset. For the overall score of 4 or 5, the sentiment is 'positive'. For the overall score of 2 or 1, the sentiment is 'negative'; if overall score equals to 3, the sentiment is 'neutral'.

In [11]:
df.loc[df['overall'] >= 4, 'sentiment'] = 'positive'
df.loc[df['overall'] <= 2, 'sentiment'] = 'negative'
df.loc[df['overall'] == 3, 'sentiment'] = 'neutral'

Check the percentage of each sentiment in the dataset.

In [12]:
df['sentiment'].value_counts(normalize=True)

positive    0.805440
neutral     0.104907
negative    0.089653
Name: sentiment, dtype: float64

We do not need the neutral sentiment in the analysis because opinions of some people may incline toward positive sentiment but they still rate the product at 3. It is also the same in case of negative-inclined opinions. So neutral sentiment is quite ambiguous.

In [13]:
train_dataset = df[df['sentiment'] != 'neutral']

In [14]:
train_dataset['sentiment'].value_counts()

positive    52116
negative     5801
Name: sentiment, dtype: int64

There is imbalance between two labels, the negative sentiment are equivalent to 10% of the positive sentiment. We will use the lesser sentiment as the number of samples.

In [15]:
sample_number = min(train_dataset['sentiment'].value_counts())
sample_number

5801

Sampling for both labels.

In [16]:
train_dataset_negative = train_dataset.loc[train_dataset['sentiment'] ==  'negative']
train_dataset_negative = train_dataset_negative.sample(n=sample_number)

train_dataset_positive = train_dataset.loc[train_dataset['sentiment'] ==  'positive']
train_dataset_positive = train_dataset_positive.sample(n=sample_number)

The numbers of positive and negative sentiment labels are equal after sampling.

In [17]:
print("The number of positive sentiment label:", len(train_dataset_positive))
print("The number of negative sentiment label:", len(train_dataset_negative))


The number of positive sentiment label: 5801
The number of negative sentiment label: 5801


# Build a Classification model using Bag-of-words method.

This will be done by the nltk library and it requires to process all the reviews into a "bag-of-words" before applying classification. To create the word bag, first we have to combine all review into one large string then we tokenize that large string to a list of words - means the bag-of-words.

Create the Positive bag-of-words.

In [18]:
#Create a list of all positive reviews.
positive_reviews = train_dataset_positive['reviewText'].tolist()

In [19]:
#The first positive review in the list.
positive_reviews[0]

"Not only is this Bungle's most insane release, but it is his most eclectic...elements of sisxties surf rock, speed metal, garage punk, Italian mob, chamber, German thrash, polka, funk, techno, jazz, Arabian, eighties experimental pop, cartoon, freak theater, ethnic, Bmovie soundtrack, and uncategorizable miscellania are seen throughout-ofcourse all jumbled together as the other reviews said. The lyrics (which you have to read since it is virtually impossible to hear them because of the unorthodox manner in which they are layed out) are usually short, but are very dememted and distorted in that, talking about Arabian religion (&quot;Desert Search For Techno Allah&quot;), insomnia (&quot;Phlegmatics&quot;), dark obsessions (&quot;Carry Stress In The Jaw&quot;, which includes a brilliant excerpt from an Edgar Allen Poe writing, and &quot;Everyone I Knew...&quot;), humourous absurdities (the truly awesome &quot;Ma Meeshka Mow Skowz&quot;, which is my favorite track, and &quot;Platypus&quo

In [20]:
print("The number of positive reviews:", len(positive_reviews))

The number of positive reviews: 5801


In [21]:
positive_text = str()

for positive_review in positive_reviews:
    #Transform a positive review into a string.
    positive_review = str(positive_review)
    
    #Lower all the words to eliminate duplicates easier later.
    positive_review = positive_review.lower()
    
    #Join all reviews into one large string.
    positive_text = ' '.join([positive_text, positive_review])

In [22]:
print(f"Datatype of positive_text is {type(positive_text)} and it has {len(positive_text)} characters!")

Datatype of positive_text is <class 'str'> and it has 6850708 characters!


Our bag-of-words does not include stop words in English, punctuations, and whitespaces.

In [23]:
#List of stop words in English.
stop_words = stopwords.words('english')
stop_words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [24]:
#List of all punctuations.
punctuations = list(string.punctuation)
punctuations

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']

In [25]:
#Create a set of stop words and punctuation.
stop_words = set(stop_words + punctuations)

In [26]:
#Tokenize the list of positive words using whitespace method.
#It mean that we will separate the words by the whitespces, tabs, or new line.
positive_words = WhitespaceTokenizer().tokenize(positive_text)

Print out the fist 50 words in the positive words list. There are many words still connected with punctuations.

In [27]:
positive_words[:50]

['not', 'only', 'is', 'this', "bungle's", 'most', 'insane', 'release,', 'but', 'it', 'is', 'his', 'most', 'eclectic...elements', 'of', 'sisxties', 'surf', 'rock,', 'speed', 'metal,', 'garage', 'punk,', 'italian', 'mob,', 'chamber,', 'german', 'thrash,', 'polka,', 'funk,', 'techno,', 'jazz,', 'arabian,', 'eighties', 'experimental', 'pop,', 'cartoon,', 'freak', 'theater,', 'ethnic,', 'bmovie', 'soundtrack,', 'and', 'uncategorizable', 'miscellania', 'are', 'seen', 'throughout-ofcourse', 'all', 'jumbled', 'together']

In [28]:
pos_words = []

for positive_word in positive_words:
    #Remove punctuations that may be still connected with the words.
    positive_word = positive_word.strip(string.punctuation)
    
    #Remove stop words and punctuations.
    if positive_word not in stop_words:
        pos_words.append(positive_word)

In [29]:
print("Number of positive words BEFORE removing stop words and puntuation:", len(positive_words))
print("Number of positive words AFTER removing stop words and puntuation:", len(pos_words))

Number of positive words BEFORE removing stop words and puntuation: 1197737
Number of positive words AFTER removing stop words and puntuation: 656486


View the most 50 common positive words.

In [30]:
positive_word_frequency = nltk.FreqDist(pos_words)
positive_word_frequency.most_common(50)

[('album', 12763), ('one', 6576), ('song', 6442), ('', 6074), ('like', 5734), ('songs', 5255), ('great', 4320), ('music', 4111), ('good', 3691), ('best', 3578), ('cd', 3502), ('love', 3113), ('track', 2668), ('first', 2650), ('time', 2594), ('really', 2431), ('get', 2276), ('still', 2194), ('sound', 2049), ('tracks', 2031), ('albums', 2009), ('would', 1991), ('also', 1990), ('well', 1990), ('even', 1912), ('rock', 1851), ('band', 1838), ('new', 1789), ('much', 1773), ('lyrics', 1700), ('way', 1658), ('back', 1655), ('better', 1537), ('classic', 1522), ('another', 1508), ('many', 1484), ('years', 1339), ('two', 1337), ('ever', 1328), ('think', 1324), ('listen', 1296), ('make', 1256), ('never', 1240), ("i'm", 1217), ('pop', 1216), ('know', 1207), ('heard', 1190), ('sounds', 1187), ('could', 1182), ('say', 1168)]

Create the Negative bag-of-words similarly to creating the Positive bag-of-words.

In [31]:
negative_reviews = train_dataset_negative['reviewText'].tolist()

negative_text = str()
for negative_review in negative_reviews:
    negative_review = str(negative_review)
    negative_review = negative_review.lower()
    negative_text = ' '.join([negative_text, negative_review])
    
negative_words = WhitespaceTokenizer().tokenize(negative_text)
    
neg_words = []
for negative_word in negative_words:
    negative_word = negative_word.strip(string.punctuation)
    if negative_word not in stop_words:
        neg_words.append(negative_word)

In [32]:
negative_word_frequency = nltk.FreqDist(neg_words)
negative_word_frequency.most_common(50)

[('album', 9085), ('like', 6279), ('songs', 3980), ('one', 3912), ('', 3641), ('song', 3638), ('good', 3331), ('music', 3069), ('cd', 2757), ('really', 2377), ('even', 2353), ('get', 2339), ('would', 1860), ('much', 1829), ('better', 1665), ('first', 1635), ('time', 1610), ('sound', 1560), ('bad', 1457), ("i'm", 1440), ('great', 1408), ('know', 1406), ('lyrics', 1406), ('best', 1402), ('tracks', 1383), ('think', 1332), ('albums', 1320), ('new', 1313), ('track', 1300), ('make', 1288), ('say', 1274), ('band', 1253), ('way', 1252), ('people', 1235), ('rap', 1231), ('sounds', 1219), ('love', 1196), ('still', 1155), ('back', 1151), ('well', 1126), ('listen', 1102), ('could', 1087), ('nothing', 1083), ('heard', 1012), ('got', 1003), ('never', 996), ('something', 977), ('two', 968), ('ever', 968), ('buy', 961)]

In [33]:
print("Number of Positive word:", len(pos_words))
print("Number of Negative word:", len(neg_words))

Number of Positive word: 656486
Number of Negative word: 488205


More specifically, we will use Naive Bayes Classifier of nltk library. This classifier requires the inputs have to be in the set format (feature, label). More details can be found in nltk documentation.

This function will transform a word in the bag to the required set format (feature, label).

In [34]:
def word_features(words):
    return dict([(word, True) for word in words.split()])

Create the list of (feature, label) sets for the positive and negative bag-of-words.

In [35]:
positive_features = []

for pos_word in pos_words:
    positive_feature = word_features(pos_word)
    positive_feature = (positive_feature, 'positive')
    positive_features.append(positive_feature)

In [36]:
negative_features = []

for neg_word in neg_words:
    negative_feature = word_features(neg_word)
    negative_feature = (negative_feature, 'negative')
    negative_features.append(negative_feature)

Final format of the positive and negative bag-of-words.

In [37]:
positive_features[:10]

[({"bungle's": True}, 'positive'), ({'insane': True}, 'positive'), ({'release': True}, 'positive'), ({'eclectic...elements': True}, 'positive'), ({'sisxties': True}, 'positive'), ({'surf': True}, 'positive'), ({'rock': True}, 'positive'), ({'speed': True}, 'positive'), ({'metal': True}, 'positive'), ({'garage': True}, 'positive')]

In [38]:
negative_features[:10]

[({'cd': True}, 'negative'), ({'boring': True}, 'negative'), ({'somewhat': True}, 'negative'), ({'original': True}, 'negative'), ({'boring': True}, 'negative'), ({'good': True}, 'negative'), ({'song': True}, 'negative'), ({'glamorous': True}, 'negative'), ({'life': True}, 'negative'), ({'downloaded': True}, 'negative')]

Combine the positive and negative bag-of-words into one big bag then shuffle all the words inside.

In [39]:
feature_label_pairs = positive_features + negative_features
random.shuffle(feature_label_pairs)

Split "the big bag" into train and test dataset with the .8 training split ratio.

In [40]:
train_ratio = 0.8
split_point = int(len(feature_label_pairs)*train_ratio)
train_set, test_set = feature_label_pairs[:split_point], feature_label_pairs[split_point:]

Train and test the model. Accuracy is used to evaluated model's performance.

In [41]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

accuracy = nltk.classify.accuracy(classifier, test_set)
accuracy = round(accuracy*100, 2)
print(f"Accuracy rate of Naive Bayes Classification model: {accuracy}%")

Accuracy rate of Naive Bayes Classification model: 62.13%
